# Install Functional.NetCDF Dependency

In [ ]:
#r "nuget: Functional.NetCDF, 0.1.2"

Error: C:\Users\mwteg\.packagemanagement\nuget\Projects\9464--a8bcfcf7-1c80-4314-aaa1-6a64bbc2d85e\Project.fsproj : error NU1101: Unable to find package Functional.NetCDF. No packages exist with this id in source(s): C:\Program Files\dotnet\sdk\6.0.300\FSharp\library-packs, Microsoft Visual Studio Offline Packages, nuget.org

# Construct and run a TimeQuery

In [ ]:
open BeardedPlatypus.Functional.NetCDF

/// <summary>
/// The <see cref="TimeQuery"/> implements the <see cref="IQuery"/> to obtain
/// the Time component of a NetCDF UGRID file.
/// It retrieves the start time and time steps.
/// </summary>
type TimeQuery () =
    let interpretUnitsString (unitsString: string) : string * DateTime =
        match unitsString.Split([| " since " |], StringSplitOptions.RemoveEmptyEntries) with
        | [| ts; dt |] -> (ts, DateTime.Parse dt)
        | _ -> failwith "Invalid units string"

    member val public StartTime : DateTime = DateTime() with get, set
    member val public TimeSteps : TimeSpan list = [] with get, set

    interface IQuery with
        member this.Execute (repository: IRepository) : unit =
            
            // Retrieve the id by finding the first component that defines a time component.
            // In the case of UGRID files, this is a variable that has a standard_name attribute
            // set to "time".
            let id = (repository.RetrieveVariablesWithAttributeWithValue ("standard_name", "time")) |> Seq.head


            // The time variable should always have a "units" attribute, that defines a string 
            // as "<time-quantity> since <date>", for example "seconds since 2001-01-01 00:00:00 +00:00"
            let unitsString = repository.RetrieveVariableAttribute (id, "units")
            let (timeStep: string, startTime: DateTime) = interpretUnitsString (unitsString.Values |> Seq.head)

            this.StartTime <- startTime

            // The "time" variable is a 1D sequence of doubles, we will convert this to time spans
            // by using the <time-quantity> obtained from the units string.
            let value : IVariableValue<double> = repository.RetrieveVariableValue<double> id

            let fTimeStep = 
                match timeStep with 
                | "seconds" -> TimeSpan.FromSeconds
                | "hours"   -> TimeSpan.FromHours
                | "days"    -> TimeSpan.FromDays
                | _         -> failwith "Invalid timestep"

            this.TimeSteps <- value.Values 
                              |> Seq.map fTimeStep
                              |> Seq.toList


Error: input.fsx (15,15)-(15,21) typecheck error The type 'IQuery' is not defined.
input.fsx (15,15)-(15,21) typecheck error The type 'obj' is not an interface type
input.fsx (15,15)-(15,21) typecheck error The type 'IQuery' is not defined.
input.fsx (15,15)-(15,21) typecheck error The type 'IQuery' is not defined.
input.fsx (16,21)-(16,28) typecheck error No abstract or interface member was found that corresponds to this override
input.fsx (16,42)-(16,53) typecheck error The type 'IRepository' is not defined.
input.fsx (16,42)-(16,53) typecheck error The type 'IRepository' is not defined.
input.fsx (21,23)-(21,73) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.
input.fsx (26,31)-(26,67) typecheck error Lookup on object of indeterminate type based on information prior to this program point. A 

In [ ]:
let query = TimeQuery()

Service.Query ("./example-data/map.nc", query :> IQuery)

// Validate the results
// Expected start time: "2001-01-01 00:00:00 +00:00"
query.StartTime